<a href="https://colab.research.google.com/github/cmd-gunma-univ/DX-highschool/blob/main/YOLO_%E5%8F%B3%E6%89%8B%E3%82%92%E6%8C%99%E3%81%91%E3%82%99%E3%81%9F%E3%82%89%E5%86%99%E7%9C%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Poseで写真撮影
例：右肩=6, 右手首=10
Y座標は下方向が +（画像は上が小さい値） →  
手首 Y < 肩 Y → 手が上がっている！と判定

In [ ]:
# ポーズ検出 + 右手検出で写真撮影

from ultralytics import YOLO
import cv2, time
from IPython.display import display, Image, clear_output
import os

model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,  640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

save_dir = "captured"
os.makedirs(save_dir, exist_ok=True)

photo_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, imgsz=640, conf=0.25, verbose=False)[0]
        annotated = results.plot()

        if results.keypoints is not None:
            kps = results.keypoints.xy.cpu().numpy()  # [n_person, 17, 2]

            for p in kps:
                right_shoulder = p[6]  # (x, y)
                right_wrist = p[10]    # (x, y)

                # ✅ Y座標が肩より上なら撮影
                if right_wrist[1] < right_shoulder[1]:
                    photo_count += 1
                    filename = f"{save_dir}/shot_{photo_count:03d}.jpg"
                    cv2.imwrite(filename, frame)
                    print(f"📸 撮影！ → {filename}")
                    time.sleep(1.0)  # 多重撮影防止

        _, jpg = cv2.imencode(".jpg", annotated)
        clear_output(wait=True)
        display(Image(data=jpg.tobytes()))
        time.sleep(0.03)

except KeyboardInterrupt:
    print("⏹ 停止しました。")

finally:
    cap.release()
    print("📸 カメラ解放完了")